In [ ]:
class gen_sentences:
  def __init__(self):
    # import spacy
    import requests as r
    from bs4 import BeautifulSoup as bs_4
    #instalar el modulo "es_core_news_sm" en caso de que no este instalado
    try:
      import es_core_news_sm
    except ModuleNotFoundError:
      !python -m spacy download es_core_news_sm
      import es_core_news_sm
    
    self.BeautifulSoup = bs_4
    self.requests = r
    self.nlp = es_core_news_sm.load()
  
  def get_ner(self,oracion=None,Lem=False):
    word_dict = {}
    word_ = []
    class_ = []
    doc = self.nlp(oracion)

    strg_lemma = ""
    if Lem == True:
      for token in doc:
        strg_lemma = strg_lemma + token.lemma_ + " "
    else:
      strg_lemma = str(doc)
    
    doc = self.nlp(strg_lemma)
    for token in doc:
      word_.append(token.text)
      class_.append(token.pos_)
    word_dict.update({"word":word_,"class":class_})
    return word_dict
  
  def get_words(self,oracion=None,Lem=False):
    doc = self.nlp(oracion)
    strg_lemma = ""
    if Lem == True:
      #Lematizamos la oración, es decir convertir las palabras a su forma base común
      for token in doc:
        strg_lemma = strg_lemma + token.lemma_ + " "
    else:
      strg_lemma = str(doc)
    #Identificar "NOUNS" "ADJ" "ADVERBS" y "VERBS"
    doc_lemma = self.nlp(strg_lemma)
    words = []
    word_class = ["VERB","NOUN","ADJ","ADV"]
    for token in doc_lemma:
      for i in word_class:
        if token.pos_ == i:
          words.append(token.text)
    return words

  def paraphrase(self,oracion=None,Lem=False):
    doc = self.nlp(oracion)
    final_sentences=[oracion.lower()]

    strg_lemma = ""
    if Lem == True:
      for token in doc:
        strg_lemma = strg_lemma + token.lemma_ + " "
    else:
      strg_lemma = str(doc)
    
    doc_lemma = self.nlp(strg_lemma)
    
    words = []
    word_class = ["VERB","NOUN","ADJ","ADV",]
    for token in doc_lemma:
      for i in word_class:
        if token.pos_ == i:
          words.append(token.text)

    
    for it_f,word in enumerate(words):
      sentences=[]
      url = 'http://www.wordreference.com/sinonimos/'
      enlace = word
      buscar = url + enlace
      resp = self.requests.get(buscar)
      bs = self.BeautifulSoup(resp.text,'html')
      lista = bs.find_all(class_ = 'trans clickable')

      
      for response in lista:
        syn = response.find('li')

      syn_array = []
      for T in syn:
        array = T.split()
        for it, value in enumerate(array):
          syn_array.append(value.replace(",",""))

    
      if it_f == 0:
        for i_, iter in enumerate(syn_array):
          final_sentences.append(strg_lemma.replace(word,iter).lower())
      if it_f > 0:
        for sentence in final_sentences:
          for iter_ in syn_array:
            sentences.append(sentence.replace(word,iter_).lower()) 
        final_sentences = final_sentences + sentences
        
    return final_sentences

In [ ]:
ner_model=gen_sentences() #Instanciamos la clase
#Solo descarga una vez el modelo "es_core_news_sm", si lo vuelves a instanciar debido a alguna modificación
#en el codigo, este no lo descargará denuevo

     |████████████████████████████████| 16.2MB 16.1MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172935 sha256=0423716badfdb0a7901fd93cc363bcc58a80b9ed4569d19199e524bc41a393fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-som9ewln/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [ ]:
#Medicion del tiempo de respuesta, esto con motivos de optimización de las funciones de la clase mas adelante
from time import time
tiempo_inicial = time()
ner_model.get_ner('hola savvy A, ¿Podrías mostrar la definición de x_1 maravillado 8 podrido lentamente ?')
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial
print(tiempo_ejecucion)

0.01903700828552246


In [ ]:
#Obtenemos la clase a la que pertenece cada palabra "VERB,NOUN,ADJ,ADV,PUNCT,PROPN,NUM,etc."
NER_=ner_model.get_ner('Hola Savvy, ¿Podrias mostrar la definición de x_1?')
for it,i in enumerate(NER_["word"]):
  print("Palabra: ",i," CLASE: >>",NER_["class"][it],"<<")

Palabra:  Hola  CLASE: >> PROPN <<
Palabra:  Savvy  CLASE: >> PROPN <<
Palabra:  ,  CLASE: >> PUNCT <<
Palabra:  ¿  CLASE: >> PUNCT <<
Palabra:  Podrias  CLASE: >> PROPN <<
Palabra:  mostrar  CLASE: >> VERB <<
Palabra:  la  CLASE: >> DET <<
Palabra:  definición  CLASE: >> NOUN <<
Palabra:  de  CLASE: >> ADP <<
Palabra:  x_1  CLASE: >> PROPN <<
Palabra:  ?  CLASE: >> PUNCT <<


In [ ]:
#Obtener las palabras que pueden ser utilizadas para desarrollar mas oraciones
#Obtener los "VERBS", "NOUNS", "ADJ" y "ADV"
ner_model.get_words('hola savvy a, ¿Podrias mostrar la definición de x_1?')

['mostrar', 'definición']

In [ ]:
#Esta funcion genera directamente nuevas oraciones a partir de los sinonimos que existan de cualquiera de las palabras
#dentro de la oracion
#ner_model.paraphrase("mostrar la definición de Capital Financiero")
ner_model.paraphrase("¿cuál es el motivo de ahorrar?")

['¿cuál es el motivo de ahorrar?',
 '¿cuál es el originar de ahorrar?',
 '¿cuál es el causar de ahorrar?',
 '¿cuál es el promover de ahorrar?',
 '¿cuál es el producir de ahorrar?',
 '¿cuál es el motivo de economizar?',
 '¿cuál es el motivo de guardar?',
 '¿cuál es el motivo de atesorar?',
 '¿cuál es el motivo de conservar?',
 '¿cuál es el motivo de reservar?',
 '¿cuál es el originar de economizar?',
 '¿cuál es el originar de guardar?',
 '¿cuál es el originar de atesorar?',
 '¿cuál es el originar de conservar?',
 '¿cuál es el originar de reservar?',
 '¿cuál es el causar de economizar?',
 '¿cuál es el causar de guardar?',
 '¿cuál es el causar de atesorar?',
 '¿cuál es el causar de conservar?',
 '¿cuál es el causar de reservar?',
 '¿cuál es el promover de economizar?',
 '¿cuál es el promover de guardar?',
 '¿cuál es el promover de atesorar?',
 '¿cuál es el promover de conservar?',
 '¿cuál es el promover de reservar?',
 '¿cuál es el producir de economizar?',
 '¿cuál es el producir de guard